In [ ]:
!pip install tensorflow matplotlib numpy scikit-learn seaborn nltk opencv-python

In [ ]:
# Object detection using Transfer Learning of CNN architectures
# a. Load in a pre-trained CNN model trained on a large dataset
# b. Freeze parameters (weights) in model’s lower convolutional layers
# c. Add custom classifier with several layers of trainable parameters to model
# d. Train classifier layers on training data available for task
# e. Fine-tune hyper parameters and unfreeze more layers as needed

In [2]:
# ============================================================
# Object Detection using Transfer Learning of CNN Architectures
# Practical Exam Solution
# ============================================================

# a. Import required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, applications
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ============================================================
# DATASET PREPARATION - CIFAR-10
# ============================================================

print("=" * 60)
print("OBJECT DETECTION WITH TRANSFER LEARNING")
print("=" * 60)

# Load CIFAR-10 dataset
print("📦 LOADING CIFAR-10 DATASET...")
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck']

print("📊 DATASET INFORMATION:")
print(f"Training data shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"Number of classes: {len(class_names)}")
print(f"Class names: {class_names}")

# Data preprocessing
def preprocess_data(images, labels):
    """
    Preprocess images for transfer learning
    """
    # Convert to float32 and normalize to [0, 1]
    images = images.astype('float32') / 255.0
    
    # Resize images to match pre-trained model input size (224x224)
    images_resized = []
    for img in images:
        img_resized = cv2.resize(img, (224, 224))
        images_resized.append(img_resized)
    
    images_resized = np.array(images_resized)
    
    # Convert labels to categorical
    labels_categorical = keras.utils.to_categorical(labels, len(class_names))
    
    return images_resized, labels_categorical

print("🔄 PREPROCESSING DATA...")
x_train_processed, y_train_processed = preprocess_data(x_train, y_train)
x_test_processed, y_test_processed = preprocess_data(x_test, y_test)

print(f"Processed training data shape: {x_train_processed.shape}")
print(f"Processed training labels shape: {y_train_processed.shape}")

# Display sample images
def display_sample_images(images, labels, class_names, num_samples=12):
    """
    Display sample images from the dataset
    """
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[i])
        plt.title(f'{class_names[np.argmax(labels[i])]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

print("\n🖼️ SAMPLE TRAINING IMAGES:")
display_sample_images(x_train_processed, y_train_processed, class_names)

# ============================================================
# a. Load pre-trained CNN model trained on large dataset
# ============================================================

print("\n" + "=" * 60)
print("STAGE A: LOAD PRE-TRAINED MODEL")
print("=" * 60)

def create_pretrained_model(base_model_name='VGG16'):
    """
    Load pre-trained CNN model
    """
    print(f"🔄 LOADING {base_model_name} PRE-TRAINED MODEL...")
    
    if base_model_name == 'VGG16':
        base_model = applications.VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
    elif base_model_name == 'ResNet50':
        base_model = applications.ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
    elif base_model_name == 'MobileNet':
        base_model = applications.MobileNet(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
    elif base_model_name == 'EfficientNetB0':
        base_model = applications.EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
    else:
        raise ValueError(f"Unsupported model: {base_model_name}")
    
    print(f"✅ {base_model_name} LOADED SUCCESSFULLY!")
    print(f"Base model input shape: {base_model.input_shape}")
    print(f"Base model output shape: {base_model.output_shape}")
    
    return base_model

# Load VGG16 as base model
base_model = create_pretrained_model('VGG16')

# ============================================================
# b. Freeze parameters in model's lower convolutional layers
# ============================================================

print("\n" + "=" * 60)
print("STAGE B: FREEZE LOWER CONVOLUTIONAL LAYERS")
print("=" * 60)

def freeze_base_model_layers(base_model, freeze_all=True):
    """
    Freeze layers in the base model
    """
    if freeze_all:
        # Freeze all layers in base model
        base_model.trainable = False
        print("❄️ ALL BASE MODEL LAYERS FROZEN")
    else:
        # Freeze only early layers
        for layer in base_model.layers[:15]:  # Freeze first 15 layers
            layer.trainable = False
        print(f"❄️ FIRST {15} LAYERS FROZEN")
    
    # Display trainable status
    trainable_count = np.sum([layer.trainable for layer in base_model.layers])
    total_count = len(base_model.layers)
    print(f"Trainable layers: {trainable_count}/{total_count}")
    
    return base_model

# Freeze all base model layers initially
base_model = freeze_base_model_layers(base_model, freeze_all=True)

# ============================================================
# c. Add custom classifier with trainable parameters
# ============================================================

print("\n" + "=" * 60)
print("STAGE C: ADD CUSTOM CLASSIFIER")
print("=" * 60)

def build_transfer_learning_model(base_model, num_classes=10):
    """
    Build complete transfer learning model with custom classifier
    """
    print("🔧 BUILDING CUSTOM CLASSIFIER...")
    
    # Create the complete model
    model = models.Sequential([
        # Base model (pre-trained, frozen)
        base_model,
        
        # Flatten the output
        layers.GlobalAveragePooling2D(),
        
        # Custom classifier layers
        layers.Dense(512, activation='relu', name='custom_fc1'),
        layers.BatchNormalization(name='custom_bn1'),
        layers.Dropout(0.5, name='custom_dropout1'),
        
        layers.Dense(256, activation='relu', name='custom_fc2'),
        layers.BatchNormalization(name='custom_bn2'),
        layers.Dropout(0.3, name='custom_dropout2'),
        
        layers.Dense(128, activation='relu', name='custom_fc3'),
        layers.BatchNormalization(name='custom_bn3'),
        
        # Output layer
        layers.Dense(num_classes, activation='softmax', name='output_layer')
    ])
    
    print("✅ CUSTOM CLASSIFIER BUILT SUCCESSFULLY!")
    return model

# Build the complete model
model = build_transfer_learning_model(base_model, len(class_names))

print("📐 MODEL ARCHITECTURE SUMMARY:")
model.summary()

# ============================================================
# d. Train classifier layers on training data
# ============================================================

print("\n" + "=" * 60)
print("STAGE D: TRAIN CLASSIFIER LAYERS")
print("=" * 60)

def compile_and_train_model(model, x_train, y_train, x_val, y_val, initial_epochs=20):
    """
    Compile and train the model
    """
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("✅ MODEL COMPILED:")
    print(f"   Optimizer: Adam (lr=0.001)")
    print(f"   Loss: Categorical Crossentropy")
    print(f"   Metrics: Accuracy")
    
    # Define callbacks
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    print(f"🚀 STARTING INITIAL TRAINING...")
    print(f"   Epochs: {initial_epochs}")
    print(f"   Training samples: {len(x_train)}")
    print(f"   Validation samples: {len(x_val)}")
    
    # Train the model
    history = model.fit(
        x_train, y_train,
        batch_size=32,
        epochs=initial_epochs,
        validation_data=(x_val, y_val),
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )
    
    print("🎯 INITIAL TRAINING COMPLETED!")
    return history, model

# Split training data for validation
from sklearn.model_selection import train_test_split
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(
    x_train_processed, y_train_processed, 
    test_size=0.2, 
    random_state=42,
    stratify=np.argmax(y_train_processed, axis=1)
)

print(f"Training split: {x_train_split.shape[0]} samples")
print(f"Validation split: {x_val_split.shape[0]} samples")

# Train the model
initial_epochs = 25
history_initial, model = compile_and_train_model(
    model, x_train_split, y_train_split, x_val_split, y_val_split, initial_epochs
)

# ============================================================
# e. Fine-tune hyperparameters and unfreeze more layers
# ============================================================

print("\n" + "=" * 60)
print("STAGE E: FINE-TUNING AND HYPERPARAMETER OPTIMIZATION")
print("=" * 60)

def fine_tune_model(model, base_model, x_train, y_train, x_val, y_val, fine_tune_epochs=15):
    """
    Fine-tune the model by unfreezing more layers
    """
    print("🔄 UNFREEZING BASE MODEL LAYERS FOR FINE-TUNING...")
    
    # Unfreeze the top layers of the base model
    base_model.trainable = True
    
    # Freeze the bottom layers and unfreeze the top layers
    fine_tune_at = len(base_model.layers) // 2  # Unfreeze top half
    
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    for layer in base_model.layers[fine_tune_at:]:
        layer.trainable = True
    
    # Count trainable layers
    trainable_count = np.sum([layer.trainable for layer in model.layers[0].layers])
    total_count = len(model.layers[0].layers)
    print(f"Fine-tuning {trainable_count}/{total_count} layers in base model")
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.0001/10),  # Lower learning rate for fine-tuning
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("✅ MODEL RECOMPILED FOR FINE-TUNING:")
    print(f"   Optimizer: Adam (lr=0.00001)")
    print(f"   Fine-tuning epochs: {fine_tune_epochs}")
    
    # Continue training
    print(f"🚀 STARTING FINE-TUNING...")
    history_fine_tune = model.fit(
        x_train, y_train,
        batch_size=32,
        epochs=fine_tune_epochs,
        initial_epoch=history_initial.epoch[-1] + 1,
        validation_data=(x_val, y_val),
        verbose=1,
        shuffle=True
    )
    
    print("🎯 FINE-TUNING COMPLETED!")
    return history_fine_tune, model

# Fine-tune the model
fine_tune_epochs = 15
history_fine_tune, model = fine_tune_model(
    model, base_model, x_train_split, y_train_split, x_val_split, y_val_split, fine_tune_epochs
)

# Combine histories for plotting
def combine_histories(initial_history, fine_tune_history):
    """
    Combine initial training and fine-tuning histories
    """
    combined_history = {}
    for key in initial_history.history.keys():
        combined_history[key] = (initial_history.history[key] + 
                               fine_tune_history.history[key])
    return combined_history

combined_history = combine_histories(history_initial, history_fine_tune)

# ============================================================
# MODEL EVALUATION AND VISUALIZATION
# ============================================================

print("\n" + "=" * 60)
print("MODEL EVALUATION AND RESULTS")
print("=" * 60)

# 1. Plot training history
def plot_training_history(history):
    """
    Plot training and validation metrics
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history['accuracy'], label='Training Accuracy', linewidth=2)
    ax1.plot(history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot loss
    ax2.plot(history['loss'], label='Training Loss', linewidth=2)
    ax2.plot(history['val_loss'], label='Validation Loss', linewidth=2)
    ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("📊 PLOTTING TRAINING HISTORY...")
plot_training_history(combined_history)

# 2. Evaluate on test set
print("🧪 EVALUATING ON TEST SET...")
test_loss, test_accuracy = model.evaluate(x_test_processed, y_test_processed, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# 3. Predictions and classification report
y_pred = model.predict(x_test_processed, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_processed, axis=1)

print("\n📈 CLASSIFICATION REPORT:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

# 4. Confusion Matrix
def plot_confusion_matrix(y_true, y_pred, class_names):
    """
    Plot confusion matrix
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix - Transfer Learning Model', fontsize=16, fontweight='bold')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

print("📋 PLOTTING CONFUSION MATRIX...")
plot_confusion_matrix(y_true_classes, y_pred_classes, class_names)

# 5. Visualize sample predictions
def visualize_predictions(images, true_labels, pred_labels, class_names, num_samples=12):
    """
    Visualize sample predictions
    """
    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        plt.subplot(3, 4, i + 1)
        plt.imshow(images[i])
        
        true_class = class_names[true_labels[i]]
        pred_class = class_names[pred_labels[i]]
        confidence = np.max(y_pred[i])
        
        color = 'green' if true_class == pred_class else 'red'
        plt.title(f'True: {true_class}\nPred: {pred_class}\nConf: {confidence:.2f}', 
                 color=color, fontsize=10)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

print("\n🔍 VISUALIZING SAMPLE PREDICTIONS...")
sample_indices = np.random.choice(len(x_test_processed), 12, replace=False)
visualize_predictions(
    x_test_processed[sample_indices],
    y_true_classes[sample_indices],
    y_pred_classes[sample_indices],
    class_names
)

# 6. Compare different pre-trained models
def compare_pretrained_models():
    """
    Compare performance of different pre-trained models
    """
    print("\n" + "=" * 60)
    print("COMPARING DIFFERENT PRE-TRAINED MODELS")
    print("=" * 60)
    
    models_to_try = ['VGG16', 'ResNet50', 'MobileNet', 'EfficientNetB0']
    results = {}
    
    for model_name in models_to_try:
        print(f"\n🧪 TESTING {model_name}...")
        
        try:
            # Load base model
            base_model = create_pretrained_model(model_name)
            base_model = freeze_base_model_layers(base_model, freeze_all=True)
            
            # Build model
            model_temp = build_transfer_learning_model(base_model, len(class_names))
            
            # Compile
            model_temp.compile(
                optimizer=Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )
            
            # Quick training for comparison
            history_temp = model_temp.fit(
                x_train_split, y_train_split,
                batch_size=32,
                epochs=5,  # Short training for comparison
                validation_data=(x_val_split, y_val_split),
                verbose=0
            )
            
            # Evaluate
            test_loss, test_accuracy = model_temp.evaluate(
                x_test_processed, y_test_processed, verbose=0
            )
            
            results[model_name] = {
                'test_accuracy': test_accuracy,
                'test_loss': test_loss,
                'val_accuracy': history_temp.history['val_accuracy'][-1]
            }
            
            print(f"✅ {model_name} - Test Accuracy: {test_accuracy:.4f}")
            
        except Exception as e:
            print(f"❌ {model_name} failed: {str(e)}")
            results[model_name] = None
    
    return results

# Compare models (commented out for speed, uncomment to run)
# print("🔄 COMPARING DIFFERENT ARCHITECTURES...")
# model_comparison = compare_pretrained_models()

# 7. Feature visualization
def visualize_feature_maps(model, sample_image, layer_name='block5_conv3'):
    """
    Visualize feature maps from convolutional layers
    """
    # Create a model that outputs the feature maps
    feature_map_model = models.Model(
        inputs=model.input,
        outputs=model.get_layer('vgg16').get_layer(layer_name).output
    )
    
    # Get feature maps
    feature_maps = feature_map_model.predict(np.expand_dims(sample_image, axis=0))
    
    # Plot feature maps
    plt.figure(figsize=(15, 10))
    plt.suptitle(f'Feature Maps - {layer_name}', fontsize=16, fontweight='bold')
    
    # Plot first 16 feature maps
    for i in range(16):
        plt.subplot(4, 4, i + 1)
        plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
        plt.axis('off')
        plt.title(f'Filter {i+1}')
    
    plt.tight_layout()
    plt.show()

print("\n🎨 VISUALIZING FEATURE MAPS...")
sample_image = x_test_processed[0]
# visualize_feature_maps(model, sample_image)  # Uncomment to see feature maps

# ============================================================
# MODEL SAVING AND DEPLOYMENT PREPARATION
# ============================================================

print("\n" + "=" * 60)
print("MODEL SAVING AND FINAL RESULTS")
print("=" * 60)

# Save the trained model
model.save('transfer_learning_object_detection.h5')
print("💾 MODEL SAVED AS: 'transfer_learning_object_detection.h5'")

# Save training history
history_df = pd.DataFrame(combined_history)
history_df.to_csv('training_history.csv', index=False)
print("💾 TRAINING HISTORY SAVED AS: 'training_history.csv'")

# Final performance summary
print("\n📊 FINAL PERFORMANCE SUMMARY:")
print("=" * 40)
print(f"Final Test Accuracy: {test_accuracy:.4f}")
print(f"Final Test Loss: {test_loss:.4f}")
print(f"Total Training Epochs: {len(combined_history['accuracy'])}")
print(f"Best Validation Accuracy: {max(combined_history['val_accuracy']):.4f}")
print(f"Vocabulary Size (Classes): {len(class_names)}")
print(f"Model Parameters: {model.count_params():,}")

# Class-wise accuracy
class_accuracy = {}
for i, class_name in enumerate(class_names):
    class_mask = y_true_classes == i
    if np.sum(class_mask) > 0:
        class_acc = np.sum(y_pred_classes[class_mask] == i) / np.sum(class_mask)
        class_accuracy[class_name] = class_acc

print(f"\n📈 CLASS-WISE ACCURACY:")
for class_name, acc in class_accuracy.items():
    print(f"  {class_name:12s}: {acc:.4f}")

print(f"\n🎯 TRANSFER LEARNING PIPELINE COMPLETED SUCCESSFULLY!")
print("=" * 60)
print("✅ Pre-trained model loaded and frozen")
print("✅ Custom classifier added and trained")
print("✅ Model fine-tuned with unfrozen layers")
print("✅ Comprehensive evaluation performed")
print("✅ Model saved for deployment")
print("✅ All stages completed successfully!")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 341)